#### This file has request-response module to get PAN images from user and convert to JSON using google VISION API response

In [9]:
# !pip install pdf2image
# !sudo apt install poppler-utils

In [1]:
import numpy as np
import cv2
from PIL import Image
import requests
import base64
import re
import PIL.ImageOps
import os
import pandas as pd
from scipy.ndimage.interpolation import zoom
import json
from PIL import ImageEnhance
import time
from pdf2image import convert_from_path
import collections
import io

In [2]:
##CALLING OCR API
def detect_image_text(image):
#     url = 'https://vision.googleapis.com/v1/images:annotate?key=xxxxxxxxx2e32e3242dasdfadxxxxxxxxxx'
# provide the key below to access google vision api
  url = 'https://vision.googleapis.com/v1/images:annotate?key='
  res = []
  img_base64 = base64.b64encode(image)
  ig = str(img_base64)
  ik=ig.replace('b\'','')
  headers={'content-type': 'application/json'}
  data ="""{
    "requests": [
      {
        "image": {
                 "content": '"""+ik[:-1]+"""'

                  },

        "features": [
          {
            "type": "DOCUMENT_TEXT_DETECTION"
          }
        ]
      }
    ]
  }"""
  r = requests.post(url, headers=headers,data=data)
  result = json.loads(r.text)
  return(result)

In [3]:
def frequency_ocr1(r):
    try:
        r['responses'][0]['textAnnotations'][1:]  
    except:
        return('0')
        
    word_infos = []
    for i, number in enumerate(r['responses'][0]['textAnnotations']):
        dic = dict()
        rect = r['responses'][0]['textAnnotations'][i]['boundingPoly']['vertices']
        text = r['responses'][0]['textAnnotations'][i]['description']
        pt1 = []
        pt2 = []
        try:
            pt1 = [rect[0]['x'], rect[0]['y']]
            pt2 = [rect[2]['x'], rect[2]['y']]
        except:
             continue
        dic['boundingBox_list'] = pt1 + pt2
        pt1.extend([-pt1[0] + pt2[0], -pt1[1] + pt2[1]])
        #str(round(pt1))
        dic['boundingBox'] = ', '.join(repr(e) for e in pt1)
        dic['text'] = text
        word_infos.append(dic)
    word_info = word_infos[1:len(word_infos)]
    urls = []
    urlls=[]
    box_cordinate_list = []
    ##########extract only text and boundingbox from dict
    for i in range(len(word_info)):
        box_cordinate_list.append(word_info[i]['boundingBox_list'])
        urls.append(word_info[i]['text'])
        urlls.append(word_info[i]['boundingBox'])

    df = pd.DataFrame({'Rows':urls, 'Co-ordinates':urlls})
    df  = pd.concat([df['Rows'],df['Co-ordinates'].str.split(",",expand= True)],axis =1)
    df.columns = ['Rows21','X','Y','Xh','Yk']
    df[['X','Y','Xh','Yk']] = df[['X','Y','Xh','Yk']].apply(pd.to_numeric)
    df['Xh'] = df['X'] + df['Xh']
    df['Yk'] = df['Y'] + df['Yk']
    return(df)

In [5]:
# To be changed according to the images location

os.chdir(r'C:\Users\Dell\Desktop\Acadgild\project\ocr\Images_and_output\Query_Images')
path = r'C:\Users\Dell\Desktop\Acadgild\project\ocr\Images_and_output\Query_Images'

In [6]:
def pan_number(df):
  pan = ""
  for i in range(0,len(df)):
    text = df.iloc[i]['Rows21']
    if(re.search("[A-Z]{5}[0-9]{4}[A-Z]",text)):
      pan = text
      return pan
  pan = "not readable"
  return pan

In [14]:
# !pip install image

In [15]:
# to covert tif image to jpg
from PIL import Image 
im = Image.open(r"C:\Users\Dell\Desktop\Acadgild\project\ocr\Images_and_output\Query_Images\000080.tif") 
im.save(r"C:\Users\Dell\Desktop\Acadgild\project\ocr\Images_and_output\Query_Images\000080.jpg", dpi=(600,600) )

In [7]:
# Below code works well only for single image for which name is given and written considering a PAN card

filename = '005066.jpg'
new_file_name = filename
image_bytes = open(new_file_name,'rb')
image_bytes = image_bytes.read()
image = np.array(Image.open(io.BytesIO(image_bytes)))
image_bytes = cv2.imencode('.jpg',image)[1].tostring()
response = detect_image_text(image_bytes)



print(response['responses'][0]['textAnnotations'][0]['description'])
temp = response['responses'][0]['textAnnotations'][0]['description']
with open("005066.txt","wb") as fp:
    fp.write(temp.encode("utf-8"))
df = frequency_ocr1(response)
print(df)
PAN_NO = pan_number(df)
print(PAN_NO)




#  NOTE: use "response" variable output to parse json

In [19]:
import ntpath
from PIL import Image 

# give folder path from where reads jpg image and converts to text 
basepath = r'C:\Users\Dell\Desktop\Acadgild\project\ocr\Images_and_output\Query_Images\\'
for val in glob.glob(basepath+'*.jpg'):
    try:
        print(ntpath.basename(val))
        filename = ntpath.basename(val).split(".")[0]

        image_bytes = open(val,'rb')
        image_bytes = image_bytes.read()
        image = np.array(Image.open(io.BytesIO(image_bytes)))
        image_bytes = cv2.imencode('.jpg',image)[1].tostring()
        # call to convert image to text using ocr
        response = detect_image_text(image_bytes)
        
        # write the respose into txt file with same name as jpg 
        try:
            text_filename = basepath+filename+".txt"
            with open(text_filename, "w") as fp:
                fp.write(response['responses'][0]['textAnnotations'][0]['description'])
                print("created file:", text_filename)
        except UnicodeEncodeError as e:
            temp = response['responses'][0]['textAnnotations'][0]['description']
            with open(text_filename,"wb") as fp1:
                fp1.write(temp.encode("utf-8"))
    except Exception as e:
        with open(basepath+"exception.txt","a") as ep:
            ep.write(filename)
#             ep.write("Exception found: {}".format(e)

162412204-000385.jpg
created file: C:\Users\Dell\Desktop\Acadgild\project\ocr\Images_and_output\Query_Images\\162412204-000385.txt
162416028-000001.jpg
162416028-000002.jpg
created file: C:\Users\Dell\Desktop\Acadgild\project\ocr\Images_and_output\Query_Images\\162416028-000002.txt
162416028-000301.jpg
created file: C:\Users\Dell\Desktop\Acadgild\project\ocr\Images_and_output\Query_Images\\162416028-000301.txt
541268689-00000001.jpg
created file: C:\Users\Dell\Desktop\Acadgild\project\ocr\Images_and_output\Query_Images\\541268689-00000001.txt
541268689-00000002.jpg
created file: C:\Users\Dell\Desktop\Acadgild\project\ocr\Images_and_output\Query_Images\\541268689-00000002.txt
541268689-00003231.jpg
created file: C:\Users\Dell\Desktop\Acadgild\project\ocr\Images_and_output\Query_Images\\541268689-00003231.txt
541268689-00003232.jpg
created file: C:\Users\Dell\Desktop\Acadgild\project\ocr\Images_and_output\Query_Images\\541268689-00003232.txt
541270491-000086.jpg
created file: C:\Users\De

#### below code to read text file write into a excel sheet with filename and text details 

In [1]:
import xlwt 
from xlwt import Workbook 

In [12]:
# Workbook is created 
wb1 = Workbook() 
  
# add_sheet is used to create sheet. 
sheet2 = wb1.add_sheet('Sheet 1') 

import ntpath

row_val = 0
basepath = r'C:\Users\Dell\Desktop\Acadgild\project\ocr\label Images\05.11.2019\\'
for val in glob.glob(basepath+'*.txt'):
    
    try:
        with open(val, "r", newline='', encoding='ISO-8859-1') as fp :
            content = fp.read()
        print(ntpath.basename(val))
        filename = ntpath.basename(val).split(".")[0]
        sheet2.write(row_val, 0, filename)
        try:
            sheet2.write(row_val, 1, content[:32767])
        except UnicodeEncodeError as e:
            sheet2.write(row_val, 1, content.encode("utf-8")[:32767])
    except Exception as e:
        print("found exception while reading {} and exception is {}".format(filename, e))
    row_val += 1
    
wb1.save('xlwtexample_main_label.xls') 

Agenda.txt
Agreement.txt
Agreement1.txt
Agreement2.txt
Business Card 1.txt
Business card.txt
Check.txt
CORPORATE 1.txt
Corporate.txt
Cover page 2.txt
Cover Page.txt
Cover Page1.txt
Diagram 1.txt
Diagram.txt
Envelope 1.txt
Envelope 2.txt
Envelope 3.txt
Envelope.txt
exception.txt
FAX.txt
FaxMailgramTelefaxTelegram 1.txt
FaxMailgramTelefaxTelegram.txt
File Folder 1.txt
File Folder.txt
Finance 1.txt
Finance.txt
Folder Back Cover 1.txt
Folder Back Cover.txt
Folder Front Cover or Spine 1.txt
Folder Front Cover or Spine.txt
Graph 1.txt
Graph 2.txt
Graph.txt
Invoice 1.txt
Invoice 2.txt
Invoice 3.txt
Invoice.txt
Memorandum 1.txt
Memorandum.txt
Memorandum3.txt
MSDSSDS.txt
Order.txt
Screenshot_1.txt
Shipping Document 1.txt
Shipping Document.txt
TAB.txt
TESTING REPORT.txt
